### VectorDatabse store vector, so we need embedding technique , to vector representation

In [125]:
from langchain.vectorstores import FAISS,Chroma,Pinecone
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.llms import GooglePalm
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')
import os
load_dotenv()

True

In [12]:
# Loading HuggingFace sentence-transformers embedding model
# HugginfaceInstructembedding in my student support chatbot

embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [121]:
query = "can i learn data science, but i am fro non-tech backgroung."
query_vector = embedding.embed_query(text=query)
print('Successfully we have generated a vector from our query')
print(type(query_vector))
print(len(query_vector))

Successfully we have generated a vector from our query
<class 'list'>
768


In [27]:
# loades csv file
loader = CSVLoader(file_path='common_question.csv', source_column="prompt", encoding='iso-8859-1')
document = loader.load()


In [28]:
print("Records in my csv : ",len(document),'\n')
document[0:5]


Records in my csv :  75 



[Document(page_content='prompt: I have never done programming in my life. Can I take this bootcamp?\nresponse: Yes, this is the perfect bootcamp for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in your current job or business using data.', metadata={'source': 'I have never done programming in my life. Can I take this bootcamp?', 'row': 0}),
 Document(page_content='prompt: Why should I trust Codebasics?\nresponse: Till now 9000 + learners have benefitted from the quality of our courses. You can check the review section and also we have attached their LinkedIn profiles so that you can connect with them and ask directly.', metadata={'source': 'Why should I trust Codebasics?', 'row': 1}),
 Document(page_content='prompt: Is there any prerequisite for taking this bootcamp ?\nresponse: Our bootcamp is specifically designed for beginners with no prior experience in this field. The only prerequisite is that you ne

In [30]:
# Documents stored in FAISS and saved at local sytem 

vector_db = FAISS.from_documents(documents=document,embedding=embedding)
vector_db.save_local("FAISS_INDEXING")

In [32]:
# To load FAISS vector database from local , pass "vector_db_name" and "embedding"
faiss_vactor_db  = FAISS.load_local("FAISS_INDEXING",embeddings=embedding,)

In [83]:
# match record with my query
user_query = "can i learn data science i am from no tech background"
response = faiss_vactor_db.similarity_search(query=user_query)
print(response[0].page_content)

prompt: I have never done programming and belong to a non-technical background. Can I take this course?
response: Yes, this is the perfect course for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in their current job or business using data.


In [85]:
# search your query from FAISS vector database
response = faiss_vactor_db.similarity_search(query="Do you provide any virtual internship?")
response

[Document(page_content='prompt: Do you provide any virtual internship?\nresponse: Yes', metadata={'source': 'Do you provide any virtual internship?', 'row': 14}),
 Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.', metadata={'source': 'Do you provide any job assistance?', 'row': 11}),
 Document(page_content='prompt: I\x92m not sure if this bootcamp is good enough for me to invest some \nmoney. What can I do?\nresponse: We got you covered. Go ahead and watch our youtube videos if you like them and want to learn further then this bootcamp is the perfect extension.', metadata={'source': 'I\x92m not sure if this bootcamp is good enough for me to invest some \nmoney. What can I do?', 'row': 4}),
 Document(page_content='prompt: Will this course guarantee me a job?\nresponse: We cre

In [45]:
# search your query from FAISS vector database WITH score
response = faiss_vactor_db.similarity_search_with_score(query="Do you provide any virtual internship?",fetch_k=2)
response

[(Document(page_content='prompt: Do you provide any virtual internship?\nresponse: Yes', metadata={'source': 'Do you provide any virtual internship?', 'row': 14}),
  0.4826004),
 (Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.', metadata={'source': 'Do you provide any job assistance?', 'row': 11}),
  1.1695364),
 (Document(page_content='prompt: I\x92m not sure if this bootcamp is good enough for me to invest some \nmoney. What can I do?\nresponse: We got you covered. Go ahead and watch our youtube videos if you like them and want to learn further then this bootcamp is the perfect extension.', metadata={'source': 'I\x92m not sure if this bootcamp is good enough for me to invest some \nmoney. What can I do?', 'row': 4}),
  1.2322673),
 (Document(page_content='prompt: Will thi

In [46]:
# Extracting information from each tuple
for document, numerical_value in response:
    print("Document Content:", document.page_content)
    print("Numerical Value:", numerical_value)
    print("Source:", document.metadata['source'])
    print("Row:", document.metadata['row'])
    print("\n")

Document Content: prompt: Do you provide any virtual internship?
response: Yes
Numerical Value: 0.4826004
Source: Do you provide any virtual internship?
Row: 14


Document Content: prompt: Do you provide any job assistance?
response: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.
Numerical Value: 1.1695364
Source: Do you provide any job assistance?
Row: 11


Document Content: prompt: Im not sure if this bootcamp is good enough for me to invest some 
money. What can I do?
response: We got you covered. Go ahead and watch our youtube videos if you like them and want to learn further then this bootcamp is the perfect extension.
Numerical Value: 1.2322673
Source: Im not sure if this bootcamp is good enough for me to invest some 
money. What can I do?
Row: 4


Document Content: prompt: Will this course guarantee me a job?
response: We created a much lig

In [48]:
# Extract information from the tuple with the highest score

max_tuple = max(response, key=lambda x: x[1])

max_document, max_numerical_value = max_tuple
print("Document Content:", max_document.page_content)
print("Numerical Value:", max_numerical_value)
print("Source:", max_document.metadata['source'])
print("Row:", max_document.metadata['row'])

Document Content: prompt: Will this course guarantee me a job?
response: We created a much lighter version of this course on YouTube available for free (click this link) and many people gave us feedback that they were able to fetch jobs (see testimonials). Now this paid course is at least 5x better than the YouTube course which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.
Numerical Value: 1.2673746
Source: Will this course guarantee me a job?
Row: 33


### FAISS use as_retriever
Now first Faiss will produce our response according to similarity,<br>
and then response will pass through llm. you can also define prompt.

In [ ]:
retriever = faiss_vactor_db.as_retriever()

In [124]:
api_key = os.getenv('GOOGLE_API_KEY')
Palm = GooglePalm(api_key=api_key)
Palm("who is the prime minister of india ?")

'Narendra Modi'

In [132]:
palm  = GooglePalm(api_key=api_key)

chain = RetrievalQA.from_chain_type(llm=Palm,
                                    chain_type="stuff",
                                    retriever=retriever,
                                    input_key="query",
                                    return_source_documents=True,
                                    )

user_query = "can i learn data science , i am from non-tech background."
response = chain(user_query)
response

{'query': 'can i learn data science , i am from non-tech background.',
 'result': 'Yes, this is the perfect bootcamp for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in your current job or business using data.',
 'source_documents': [Document(page_content='prompt: I have never done programming and belong to a non-technical background. Can I take this course?\nresponse: Yes, this is the perfect course for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in their current job or business using data.', metadata={'source': 'I have never done programming and belong to a non-technical background. Can I take this course?', 'row': 24}),
  Document(page_content='prompt: I have zero knowledge of Excel and belong to a non-technical\n background. Can I take this course?\nresponse: Yes, this is the perfect course for anyone who has never work

In [130]:
print(response['result'])

Yes, this is the perfect course for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in your current job or business using data.


THANK YOU 